In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('Lekagul Sensor Data.csv', sep = ',')

In [4]:
df

,Timestamp,car-id,car-type,gate-name
0,5/1/15 0:43,20154301124328-262,4,entrance3
1,5/1/15 1:03,20154301124328-262,4,general-gate1
2,5/1/15 1:06,20154301124328-262,4,ranger-stop2
3,5/1/15 1:09,20154301124328-262,4,ranger-stop0
4,5/1/15 1:12,20154301124328-262,4,general-gate2
...,...,...,...,...
171472,5/31/16 23:40,20161031111001-854,6,ranger-stop2
171473,5/31/16 23:42,20165831105856-579,1,general-gate4
171474,5/31/16 23:43,20161031111001-854,6,general-gate1
171475,5/31/16 23:49,20165831105856-579,1,general-gate7


In [8]:
df["gate-name"].replace(r'(entrance\d*)' , 'entrance', regex=True, inplace=True)
df["gate-name"].replace(r'(general-gate\d*)' , 'general gate', regex=True, inplace=True)
df["gate-name"].replace(r'(ranger-stop\d*)' , 'ranger-stop', regex=True, inplace=True)
df["gate-name"].replace(r'(camping\d*)' , 'camping', regex=True, inplace=True)
df["gate-name"].replace(r'(gate\d*)' , 'gate', regex=True, inplace=True)



In [11]:
df = df[['car-id', 'gate-name', 'Timestamp', 'car-type']]
df

,car-id,gate-name,Timestamp,car-type
0,20154301124328-262,entrance,5/1/15 0:43,4
1,20154301124328-262,general gate,5/1/15 1:03,4
2,20154301124328-262,ranger-stop,5/1/15 1:06,4
3,20154301124328-262,ranger-stop,5/1/15 1:09,4
4,20154301124328-262,general gate,5/1/15 1:12,4
...,...,...,...,...
171472,20161031111001-854,ranger-stop,5/31/16 23:40,6
171473,20165831105856-579,general gate,5/31/16 23:42,1
171474,20161031111001-854,general gate,5/31/16 23:43,6
171475,20165831105856-579,general gate,5/31/16 23:49,1


In [12]:
df.to_csv('cleaned-eventflow.txt',index=False,header=False,sep="\t")